In [11]:
# !pip install --quiet pytorch-lightning
import sys
sys.path.append("..")

In [12]:
from predpy.dataset import TimeSeriesRecordsDataset, SingleTimeSeriesDataset
from predpy.data_module import TimeSeriesModule
from predpy.experimentator import Experimentator, DatasetParams, ModelParams
from predpy.preprocessing import load_and_preprocess, set_index, scale
from predpy.preprocessing import moving_average
from predpy.trainer import CheckpointParams, TrainerParams, EarlyStoppingParams, LoggerParams

import pickle
from sklearn.preprocessing import MinMaxScaler
from tsai.models import TCN, ResNet, TST, RNN, TransformerModel, FCN
import pandas as pd
import seaborn as sns

In [13]:
# import numpy as np
# tmp = tmp.drop(tmp[tmp["Global_active_power"]=="?"].index)
# tmp = tmp[["Global_active_power"]]

# tmp.astype(float)
# sns.scatterplot(data=tmp, x=tmp.index, y="Global_active_power")
# tmp.head()

In [14]:
chp_p = CheckpointParams(
    dirpath="../checkpoints", monitor='val_loss', verbose=True,
    save_top_k=1)
tr_p = TrainerParams(
    max_epochs=3, gpus=1, auto_lr_find=True, overfit_batches=0.1)
es_p = EarlyStoppingParams(
    monitor='val_loss', patience=3, verbose=True)

In [15]:
models_params = [
    ModelParams(
        name_="ResNet", cls_=ResNet.ResNet, init_params={"c_in": 2, "c_out": 1}),
    ModelParams(
        name_="LSTM_h200_l1", cls_=RNN.LSTM, init_params={"c_in": 2, "c_out": 1, "hidden_size": 200, "n_layers": 1})
]
datasets_params = [
    DatasetParams(
        path="../data/daily-min-temperatures.csv",
        target="Temp",
        split_proportions=[0.8, 0.1, 0.1],
        window_size=366,
        batch_size=64,
        DatasetCls=SingleTimeSeriesDataset,
        pipeline=[
            (set_index, {"column_name": "Date"}),
            (scale, {"training_fraction": 0.8, "scaler": MinMaxScaler()}),
            (moving_average, {"window_size": 20, "col_names": ["Temp"]})
        ])
]
exp = Experimentator(models_params, datasets_params)

In [16]:
exp.run_experiments("../lightning_logs", tr_p, chp_p, es_p, experiments_path="../saved_experiments")

Global seed set to 42
/home/stachu/anaconda3/envs/dl/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning:

Checkpoint directory ../checkpoints/daily-min-temperatures/ResNet exists and is not empty.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/stachu/anaconda3/envs/dl/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning:

DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params
--------------------------------------
0 | model     | ResNet  | 478 K 
1 | criterion | MSELoss | 0     
--------------------------------------
478 K     Trainable params
0         Non-trainable params
478 K     Total params
1.915     Total estimated model params size (MB)


Global seed set to 42


Epoch 0:   0%|          | 0/8 [00:00<?, ?it/s] 

/home/stachu/anaconda3/envs/dl/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:326: UserWarning:

The number of training samples (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.



Epoch 0: 100%|██████████| 8/8 [00:00<00:00, 11.77it/s, loss=0.0486, v_num=4:09, train_loss=0.0316, val_loss=0.156]

...

    self._shutdown_workers()  File "/home/stachu/anaconda3/envs/dl/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers

    
w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)  File "/home/stachu/anaconda3/envs/dl/lib/python3.8/multiprocessing/process.py", line 147, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f37e442e550>
Exception ignored in: Traceback (most recent call last):
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f37e442e550>  File "/home/stachu/anaconda3/envs/dl/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__

Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f37e442e550>
    Traceback (most recent call last):
self._shutdown_workers()  File "/home/stachu/anaconda3/envs/dl/l

Epoch 2: 100%|██████████| 8/8 [00:00<00:00, 11.66it/s, loss=0.16, v_num=4:09, train_loss=0.0795, val_loss=0.145]

Metric val_loss improved by 0.009 >= min_delta = 0.0. New best score: 0.145
Metric val_loss improved by 0.009 >= min_delta = 0.0. New best score: 0.145
Epoch 2, global step 11: val_loss reached 0.14515 (best 0.14515), saving model to "/home/stachu/Projects/Anomaly_detection/Forecasting_models/checkpoints/daily-min-temperatures/ResNet/2021-11-07_17:24:09-v1.ckpt" as top 1
Epoch 2, global step 11: val_loss reached 0.14515 (best 0.14515), saving model to "/home/stachu/Projects/Anomaly_detection/Forecasting_models/checkpoints/daily-min-temperatures/LSTM_h200_l1/2021-11-07_17:24:09.ckpt" as top 1


Epoch 2: 100%|██████████| 8/8 [00:00<00:00, 11.13it/s, loss=0.16, v_num=4:09, train_loss=0.0795, val_loss=0.145]


/home/stachu/anaconda3/envs/dl/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning:

DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f37e442e550>
Traceback (most recent call last):
  File "/home/stachu/anaconda3/envs/dl/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/home/stachu/anaconda3/envs/dl/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/stachu/anaconda3/envs/dl/lib/python3.8/multiprocessing/process.py", line 147, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _MultiP

In [17]:
# exp = Experimentator.load_experimentator("../saved_experiments/2021-11-07_17:21:16.pkl")
# exp = Experimentator.load_experimentator("../saved_experiments/2021-11-07_16:58:09.pkl")
exp.plot_predictions(0)

In [9]:
None=1

SyntaxError: cannot assign to None (<ipython-input-9-c50d50884aea>, line 1)

In [ ]:
# !pip install --quiet tqdm

In [ ]:
# !pip install --quiet yfinance

In [ ]:
# !pip install --quiet tsai

In [ ]:
!nvidia-smi

Sun Oct 31 14:16:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1060    Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   49C    P0    26W /  N/A |    337MiB /  6078MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
import yfinance as yf
from typing import List, Dict, Union, Callable
from datetime import datetime, timedelta

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
# from pytorch_lightning.loggers import WandbLogger

import torch
from torch import autograd, nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tsai.models import TCN, ResNet, TST, RNN, TransformerModel, FCN

from IPython.display import display

import sys
sys.path.insert(0,'../src')
from dataModule.Data import scale, create_sequences
from dataModule.Data import SingleTimeSeriesDataset, SingleTimeSeriesModule
from experimentsModule.Predictor import Predictor

In [ ]:
use_cuda = torch.cuda.is_available()

if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:', torch.cuda.get_device_properties(0).total_memory/1e9)

__CUDNN VERSION: 7605
__Number CUDA Devices: 1
__CUDA Device Name: GeForce GTX 1060
__CUDA Device Total Memory [GB]: 6.373572608


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

# HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

# sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

tqdm.pandas()

pl.seed_everything(42);

Global seed set to 42


In [ ]:
N_EPOCHS = 8
BATCH_SIZE = 64
SEQ_LEN = 366
SPLIT_PROPORTIONS = [0.8, 0.1, 0.1]
TARGET = "Temp"

df = pd.read_csv("../data/daily-min-temperatures.csv").set_index("Date")
df.index = pd.to_datetime(df.index)

scaler, df = scale(df, SPLIT_PROPORTIONS[0])
# df["moving_average"] = df.rolling(20).mean()
# df["moving_average"].iloc[:20] = df["Temp"].iloc[:20]
df = df.dropna()

data_module = SingleTimeSeriesModule(df, SPLIT_PROPORTIONS, window_size=SEQ_LEN,
                                     batch_size=BATCH_SIZE)

In [ ]:
# BTC = yf.Ticker("BTC-USD")
# # df = BTC.history(period="max", interval="1d", auto_adjust=False, actions=False)
# df = BTC.history(interval="1d", start="2014-09-17", end="2021-10-16", actions=False)
# df["Open-Close-diff"] = df.apply(lambda x: x["Close"] - x["Open"], axis=1)

# from datetime import datetime
# end = datetime.strptime("06-01-2017", '%m-%d-%Y')
# df = df.loc[:end]  # remove last years due to a change behaviour
# df = df[["Close", "Open-Close-diff"]]